# Data Preparation and Process using SageMaker Processing job with Snowflake



## Introduction

Generally, when working with SageMaker processing jobs, the input data is expected to be stored in Amazon S3 and the SageMaker platform will handle data downloading from S3 and save the output results back to S3. In this way, users can achieve better lineage between the input data and output results, which also benefits the reproducability and auditability of the job. However, sometimes users would like to directly read data from their data storage without saving the data to S3 as an additional step for the processing job. In fact, it is possible to bypass the S3 data input for the processing jobs and directly read the data from other data storage given the proper connector is available. 

In this notebook, we will demonstrate how to create a SageMaker processing job to directly read data from Snowflake via the public url and perform data preprocessing and model training in one processing job. 

## Preparation


In [ ]:
import sagemaker
import boto3
import re
import os
from sagemaker import get_execution_role

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.utils import name_from_base
from sagemaker.xgboost import XGBoostProcessor

bucket=sagemaker.Session().default_bucket()
prefix = 'snowflake-sagemaker'

role = get_execution_role()

## Data processing with Amazon SageMaker Processing

Amazon SageMaker Processing allows you to run steps for data pre- or post-processing, feature engineering, data validation, or model evaluation workloads on Amazon SageMaker. Processing jobs accept data from Amazon S3 as input and store data into Amazon S3 as output.

![processing](https://sagemaker.readthedocs.io/en/stable/_images/amazon_sagemaker_processing_image1.png)

Here, we'll import the dataset and transform it with [SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html), which can be used to process terabytes of data in a SageMaker-managed cluster separate from the instance running your notebook server. In a typical SageMaker workflow, notebooks are only used for prototyping and can be run on relatively inexpensive and less powerful instances, while processing, training and model hosting tasks are run on separate, more powerful SageMaker-managed instances.  SageMaker Processing includes off-the-shelf support for Scikit-learn, various machine learning frameworks, as well as a Bring Your Own Container option, so it can be used with many different data transformation technologies and tasks.    

To use SageMaker Processing, simply supply a Python data preprocessing script as shown below.  For this example, we're using a SageMaker prebuilt Scikit-learn container, which includes many common functions for processing data.  

## Snowflake
Snowflake is an [AWS Partner](https://partners.amazonaws.com/partners/001E000000d8qQcIAI/Snowflake) with multiple AWS accreditations including AWS Competencies in Machine Learning and Data and Analytics.

In this example, we assume you have already setup snowflake account following the instruction provided in the Prerequisites_Lab_Snowflake_SageMaker lab guide and loaded the data to the snowflake data warehouse. To connect to the snowflake server, you need to install the snowflake connector in the containers that runs the processing script. In this example, we will use the `requirements.txt` file to specify the connector python library needed for the job and use the prebuilt scikit-learn container.

## SageMaker Processing job with Framework Processor

Before starting the SageMaker Processing job, we instantiate a [`FrameworkProcessor` object](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks.html).  This object allows you to specify the instance type to use in the job, as well as how many instances.

SageMaker processing job will launch the cluster and run the provide script. Based on the funtions defined in the script, we can directly read the data from the snowflake table and perform data preprocessing and even model training using XGBoost. More details of the [XGBoost Framework Processor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-xgboost.html) can be found in the document.



In [ ]:

base_job_name = 'snowflake-sagemaker-processing'

#Initialize the XGBoostProcessor
xgb = XGBoostProcessor(
    framework_version='1.2-2',
    role=get_execution_role(),
    instance_type='ml.m5.xlarge',
    instance_count=1,
    base_job_name=base_job_name,
)

processing_job_name = name_from_base(base_job_name)

In [ ]:
#Run the processing job
xgb.run(
    code='processing_data.py',
    source_dir='code',
    arguments = [
                 '--model_dir', '/opt/ml/processing/output/model',
                 '--num_round', '6',
                 '--K', '5'
                ],
    outputs=[
        ProcessingOutput(output_name="model", source="/opt/ml/processing/output/model"),
    ],
    job_name=processing_job_name,
)